# SPA - J = 0 to 1
Simulating the SPA1, i.e. transferring molecules from J = 0 to J = 0 using microwave driven adiabatic passage.

## Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

from itertools import product
from pathlib import Path
import random
import string
import time


from joblib import Parallel, delayed
import matplotlib.pyplot as plt
plt.style.use(['seaborn-darkgrid', 'thesis'])
# # plt.rcParams["figure.figsize"] = (16, 9)

# # plt.style.use(['fivethirtyeight'])
# # Font for labels
# plt.rcParams['font.family'] = 'serif'
# # Linewidths for plots
# plt.rcParams['lines.linewidth'] = 1.6
# plt.rcParams['lines.marker'] = ''
# # Axes
# plt.rcParams['axes.labelsize'] = 18
# plt.rcParams['axes.labelweight'] = 'bold'
# plt.rcParams['axes.grid'] = True
# # Ticks
# plt.rcParams['xtick.labelsize'] = 16
# plt.rcParams['ytick.labelsize'] = 16
# # Legend
# plt.rcParams['legend.fontsize'] = 16

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import seaborn as sns

import centrex_TlF
from centrex_TlF import UncoupledBasisState
from centrex_TlF.transitions import calculate_transition_frequency
from state_prep.electric_fields import ElectricField, E_field_ring, Ez_from_csv, Ez_from_csv_offset
from state_prep.hamiltonians import SlowHamiltonian, SlowHamiltonianOld
from state_prep.intensity_profiles import GaussianBeam, MeasuredBeam, BackgroundField
from state_prep.magnetic_fields import MagneticField
from state_prep.microwaves import Polarization, Intensity, MicrowaveField
from state_prep.plotters import CouplingPlotter
from state_prep.simulator import Simulator
from state_prep.trajectory import Trajectory
from state_prep.utils import vector_to_state

## Define molecule trajectory
Defining the molecule trajectory as a Trajectory object. Specifying the initial position as z = 50 mm, i.e. 50mm  after the center of the microwaves is, and on the axis of the beamline. Taking the beam velocity to be 200 m/s in the z-direction. The simulation ends when `zfin` is reached, 200 mm from the microwwaves.

In [ ]:
trajectory = Trajectory(Rini = np.array((0,0,-75e-3)), Vini = np.array((0,0,184.)), zfin = 200e-3)

## Define electric field
Defining an electric field that is an approximation of what the molecules should see in the real experiment.

In [ ]:
# #Define positions for centers of ring electrodes (in m)
# electrode_offset = 0.0
# z1 = -85.725e-3  + electrode_offset
# z2 = 85.725e-3 + electrode_offset

# #Define voltages on the rings
# V1 = 5e3*172/155
# V2 = V1*0.07

# #Define a quickly decaying function to simulate adding shielding between state prep region and detection region
# decay_function = lambda R, Z0, l: np.exp(-0.5*((R[2]-Z0)*float(R[2]>Z0)/l)**2)
# decay_tanh = lambda R,Z0,l,z_shield: .5*(1-np.tanh((R[2]-Z0)/l))

# #Define electric field as function of position
# z_shield = 0.085
# l_decay = 0.02
# z0 = 116.4/1e3

# E_R = lambda R: ((E_field_ring(R, z0 = z1, V = V1)
#                         + E_field_ring(R, z0 = z2, V = V2))*decay_tanh(R,z0,l_decay,z_shield))


# Define electric field based on finite element simulations
Ez = Ez_from_csv()
# Ez = Ez_from_csv()
E_R = lambda R: np.array([0,0,Ez(R[2])])*263/263


electric_field = ElectricField(E_R, trajectory.R_t)

In [ ]:
# Plot the electric field as a check:
ts, Es, ax = electric_field.plot(trajectory, position = True)

print(f"Max electric field: {np.max(Es):.2f} V/cm")
print(f"Electric field at z = 0: {E_R(np.array([0,0,0]))[2]:.2f} V/cm")

## Define magnetic field
Define the magnetic field. Using a constant field, either a nominal small field (so mF states become non-degenerate eigenstates of the Hamiltonian) or something resembilng Earth's magnetic field.

In [ ]:
B = np.array((0,0,1e-4))
# B = np.array((0.2,0.45,0.1))
def B_R(R):
    if len(R.shape) == 1:
        return B
    else:
        return B.reshape((3,1)) * np.ones(R.shape)

magnetic_field = MagneticField(B_R, R_t = trajectory.R_t)

In [ ]:
magnetic_field.plot(trajectory)

## Initial and final states
Define approximate initial states here.

In [ ]:
#Define some useful approximate states
J0_singlet = 1/np.sqrt(2) * (1*UncoupledBasisState(J = 0, mJ = 0, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = -1/2, P = +1, Omega = 0, electronic_state='X')
                                       -1* UncoupledBasisState(J = 0, mJ = 0, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = 1/2, P = +1, Omega = 0, electronic_state='X'))

J0_triplet_0 = 1/np.sqrt(2) * (1*UncoupledBasisState(J = 0, mJ = 0, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = -1/2, P = +1, Omega = 0, electronic_state='X')
                            +1* UncoupledBasisState(J = 0, mJ = 0, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = 1/2, P = +1, Omega = 0, electronic_state='X'))

J0_triplet_p = 1*UncoupledBasisState(J = 0, mJ = 0, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = 1/2, P = +1, Omega = 0, electronic_state='X')

J0_triplet_m = 1*UncoupledBasisState(J = 0, mJ = 0, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = -1/2, P = +1, Omega = 0, electronic_state='X')

initial_states_approx = [J0_singlet, J0_triplet_m, J0_triplet_0, J0_triplet_p]

In [ ]:
#Define some useful approximate states
J1_singlet = 1/np.sqrt(2) * (1*UncoupledBasisState(J = 1, mJ = 0, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = -1/2, P = -1, Omega = 0, electronic_state='X')
                                       -1* UncoupledBasisState(J = 1, mJ = 0, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = 1/2, P = -1, Omega = 0, electronic_state='X'))

J1_triplet_0 = 1/np.sqrt(2) * (1*UncoupledBasisState(J = 1, mJ = 0, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = -1/2, P = -1, Omega = 0, electronic_state='X')
                            +1* UncoupledBasisState(J = 1, mJ = 0, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = 1/2, P = -1, Omega = 0, electronic_state='X'))

J1_triplet_p = 1*UncoupledBasisState(J = 1, mJ = 0, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = 1/2, P = -1, Omega = 0, electronic_state='X')

J1_triplet_m = 1*UncoupledBasisState(J = 1, mJ = 0, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = -1/2, P = -1, Omega = 0, electronic_state='X')

J1_ppp = 1*UncoupledBasisState(J = 1, mJ = 1, I1 = 1/2, m1 = 1/2, I2 = 1/2, m2 = 1/2, P = -1, Omega = 0, electronic_state='X')

J1_pmp = 1*UncoupledBasisState(J = 1, mJ = 1, I1 = 1/2, m1 = -1/2, I2 = 1/2, m2 = 1/2, P = -1, Omega = 0, electronic_state='X') 


final_states_approx = [J1_singlet, J1_triplet_m, J1_triplet_0, J1_triplet_p]

## Define slowly varying Hamiltonian
Define a Hamiltonian object that represents the slowly varying part of the Hamiltonian (i.e. everything except the microwaves). Using J = 0-3 here since we're interested in the time-evolution of J = 0 and 1 states; J = 0 and 3 are absolutely required since the electric field couples J = 0 to these, J = 3 is not strictly necessary. Since the Stark energies are << the rotational energies, we mostly just need the J of interest $\pm 1$.

In [ ]:
Js = [0,1,2,3]
hamiltonian = SlowHamiltonian(Js, trajectory, electric_field, magnetic_field)
# hamiltonian = SlowHamiltonianOld(0, 3, trajectory, electric_field, magnetic_field, '../hamiltonians/TlF_X_state_hamiltonian0to3_2020_03_03.pickle')

## Define microwave Hamiltonian
We will define a microwave field with the following properties
- A Gaussian spatial profile with 1'' intensity FWHM (cylindrically symmetric) 
- propagating along the X-direction
- polarized along the Z-direction
- Calculate power based on the desired Rabi rate for the desired transition

### J = 0 to 1 microwaves

#### Define states

In [ ]:
state1 = J0_singlet
state2 = J1_singlet

#### Center position

In [ ]:
R0 = np.array((0.00,0,-0.0))

In [ ]:
R0.shape

#### Polarization:

In [ ]:
# Define polarization components
p_x = np.array([1, 0, 0])
p_y = np.array([0, 1, 0])
p_z = np.array([0, 0, 1])

# Define a function that gives the main component of the polarization as a function of position
p_main = p_z + 0.0 * p_y
P_R = lambda R: p_main/np.sqrt(np.sum(p_main**2))

# Define k-vector
k = np.array((1,0,0))

pol = Polarization(P_R, k, f_long = 0)

#### Frequency
Calculate the microwave frequency between specified states at a specific point

In [ ]:
muW_freq = calculate_transition_frequency(state1, state2, hamiltonian.H_R(R0), hamiltonian.QN)

In [ ]:
print(f"Transition frequency at center of microwaves: {muW_freq/1e9:.5f} GHz")

#### Intensity profile:

In [ ]:
# Define a Gaussian intensity profile
# intensity = MeasuredBeam(power = 1e-3, sigma = 1*25.4e-3/(2*np.sqrt(2*np.log(2))), R0 = R0, k = k, freq = muW_freq)
intensity = GaussianBeam(power = 1e-3, sigma = 1*25.4e-3/(2*np.sqrt(2*np.log(2))), R0 = R0, k = k, freq = muW_freq)

#### Define MicrowaveField

In [ ]:
mf01 = MicrowaveField(0, 1, intensity, pol, muW_freq, hamiltonian.QN) 

### Define list of microwave fields

In [ ]:
microwave_fields = [mf01]

## Time-evolution
Next define a Simulator and use it to numerically integrate the Schroedinger equation to see how the states evolve as the electric field is ramped down. 

In [ ]:
simulator = Simulator(trajectory, electric_field, magnetic_field, initial_states_approx, hamiltonian, microwave_fields)

In [ ]:
# Set microwave parameters
## Center postion
R0 = np.array([0.00,0,-0.0])

## Sigma
mf01.intensity.sigma = 1.0*25.4e-3/(2*np.sqrt(2*np.log(2)))

## Frequency
freq = calculate_transition_frequency(state1, state2, hamiltonian.H_R(R0), hamiltonian.QN)
detuning = -0e6
mf01.set_frequency(freq + detuning)
mf01.set_position(R0)

## Power
# mf01.calculate_microwave_power(state1, state2, 2*np.pi*100e3, R0)
mf01.set_power(2e-4)


# Molecule parameters
trajectory.Rini = np.array([0,0.00,-70e-3])
trajectory.Vini = np.array([0,0,184])

In [ ]:
result = simulator.run(N_steps = 10000)

## Plot results

In [ ]:
initial_state = result.initial_states[0]
large_prob_states = result.find_large_prob_states(initial_state, N = 6)
fig, (ax, ax2) = plt.subplots(2,1, figsize = (16,18))
result.plot_state_probabilities(large_prob_states, initial_state, ax = ax, position = True)
ax.legend(fontsize = 14, bbox_to_anchor = (1.,1))
ax.axvline(x = R0[2]*1e2, ls ='--', c = 'k')
ax.set_title(f"z_offset = ${R0[2]:.2f}$ m, power = ${mf01.intensity.power:.3E}$ W")
ax.grid()

_ = electric_field.plot(trajectory, position=True, ax = ax2)

### Plot detunings
Plot the detunings for J = 0 to 1 for different $\Delta m_J$

In [ ]:
# Find energies
E_J0 = result.get_state_energy(J0_singlet) / (2*np.pi)
E_J1_singlet = result.get_state_energy(J1_singlet) / (2*np.pi)
E_J1_pmp = result.get_state_energy(J1_pmp) / (2*np.pi)

# Find transitions frequencies
freq_mJ0 = (E_J1_singlet - E_J0)/1e6
freq_mJ1 = (E_J1_pmp - E_J0)/1e6

# Get positions
zs = result.z_array*100

# Define frequency at z = 0
freq_0 = 13340

# Plot
fig, ax = plt.subplots()
ax.plot(zs, freq_mJ0-freq_0, label = r"$|J = 0, m_J = 0\rangle \rightarrow |J = 1, m_J = 0\rangle$")
ax.plot(zs, freq_mJ1-freq_0, label = r"$|J = 0, m_J = 0\rangle \rightarrow |J = 1, |m_J| = 1\rangle$")
ax.set_xlabel('Z-position / cm')
ax.set_ylabel(f'Frequency - {freq_0} MHz / MHz')
ax.axhline(0,ls='--', c='k', label = r'$\Delta = 0$')
ax.legend()

# plt.savefig(Path('G:\My Drive\CeNTREX Oskari\State preparation\SPA\Report\State preparation and diabatic transitions\Figures\EV_J01/J01_transition_freq_vs_position.pdf'))
plt.savefig("../../../figures/SPA/SPA_transition_frequency_vs_position.pdf")

### Plot intensity profile
Check that the longitudinal polarization appears correctly

In [ ]:
t_array = np.linspace(0, trajectory.get_T(), 1000)
Rs = np.array([trajectory.R_t(t) for t in t_array])
intensities_R = np.array([mf01.intensity.I_R(trajectory.R_t(t)) for t in t_array])
intensities = np.array([mf01.intensity.I_R(np.array([0.0,0.0127,z])) for z in Rs[:,2]])

fig, ax = plt.subplots()
ax.plot(Rs[:,2],intensities_R)
ax.plot(Rs[:,2],intensities)
ax.set_xlabel("z-position / m")
ax.set_ylabel(r"Intensity / $W/m^2$")
ax.axvline(x = R0[2], ls ='--', c = 'k')

In [ ]:
E_mags_R = np.array([mf01.intensity.E_R(trajectory.R_t(t)) for t in t_array])
E_mags = np.array([mf01.intensity.E_R(np.array((0.,0.0,z))) for z in Rs[:,2]])

fig, ax = plt.subplots()
ax.plot(Rs[:,2], E_mags_R)
# ax.plot(Rs[:,2], E_mags)
ax.set_xlabel("z-position / m")
ax.set_ylabel(r"Electric field / $V/cm$")
ax.axvline(x = R0[2], ls ='--', c = 'k')

### Plot polarization components as function of position
Check that the longitudinal polarization appears correctly

In [ ]:
pols = np.array([mf01.polarization.p_R(trajectory.R_t(t), mf01.intensity, freq=muW_freq) for t in t_array])

fig, ax = plt.subplots()
ax.plot(Rs[:,2], np.imag(pols[:,0]), label='p_x')
ax.plot(Rs[:,2], pols[:,1], label='p_y')
ax.plot(Rs[:,2], np.real(pols[:,2]), label='p_z')
ax.legend()
ax.set_xlabel("z-position / m")
ax.set_ylabel("Polarization component")
ax.axvline(x = R0[2], ls ='--', c = 'k')

## Parameter scans
### Helper functions
Define two helper function to run the power scans:

In [ ]:
def run_simulation(power: float, detuning: float, R0: np.ndarray, Rini:np.ndarray, Vini: np.ndarray, initial_state_idx: int, target_state: centrex_TlF.State, save_path: Path = None, N_steps = 10000):
    """
    Runs the simulation for one value of power and finds the probability of ending up in the target state.
    """
    ## Microwave settings    
    # Set position
    mf01.set_position(R0)
    
    # Set frequency
    freq = calculate_transition_frequency(state1, state2, hamiltonian.H_R(R0), hamiltonian.QN)
    mf01.set_frequency(freq + detuning)
    mf01.set_position(R0)

    # Set power
    mf01.set_power(power)
    
    # Trajectory settings
    trajectory.Rini = Rini
    trajectory.Vini = Vini
    
    # Run the simulator
    result = simulator.run(N_steps = N_steps)
    
    # Save result if save path provided
    if save_path:
        result.save_to_pickle(save_path)
    
    # Determine probability to be in initial state
    prob_ini = result.get_state_probability(result.initial_states[initial_state_idx], initial_state = result.initial_states[initial_state_idx])[-1]

    # Determine probability to be in target state
    prob_fin = result.get_state_probability(target_state, initial_state = result.initial_states[initial_state_idx])[-1]
    
    # Return a dataframe
    df = pd.DataFrame({"power":[power], "detuning":[detuning], "muw_position": [R0], "Rini":[Rini], "Vini":[Vini],  "prob_ini":[prob_ini], "prob_fin": [prob_fin], "save_path": [save_path]})
    
    return df

In [ ]:
# Define the function to be run in parallel
def parallel_func(power = 10**(-4.1), detuning = 0, R0 = np.array([0.00,0,-0.0]), Rini = np.array((0,0,-80e-3)), Vini = np.array((0,0,184.)), save = False, N_steps = 10000):
    save_path = SAVE_DIR / Path(f"result_{generate_random_key()}") if save else None
    return run_simulation(power, detuning, R0, Rini, Vini, 0, J1_singlet, save_path=save_path, N_steps = N_steps)

In [ ]:
# Function for generating random keys for the results
def generate_random_key(length: int = 6):
    return ''.join(random.choices(string.ascii_letters + string.digits, k = length))

In [ ]:
run_simulation(mf01.intensity.power, 0, np.array([0,0,-0.02]), trajectory.Rini, trajectory.Vini, 0, J1_singlet)

### Save directory

In [ ]:
# SAVE_DIR = Path('../results/SPA1 - parameter scans')

### Power values

In [ ]:
powers = 10**((np.linspace(-50,10,36)-30)/10) # in W

### Run power scan

In [ ]:
# Initialize parallel loop
df = pd.concat(Parallel(n_jobs = 9, verbose = 1)(delayed(parallel_func)(power=power, detuning = 0e6,) for power in powers)).reset_index(drop=True)

In [ ]:
fig, ax = plt.subplots()
ax.plot(10*np.log10(df.power)+30, df.prob_fin, label = r'$|J=0, F = 0\rangle$')
ax.plot(10*np.log10(df.power)+30, df.prob_ini, label = r'$|J=1, m_J = 0, I_{tot} = 0\rangle$')
ax.set_xlabel("Microwave power / dBm")
ax.set_ylabel("SPA1 efficiency")
ax.legend()
plt.savefig('../../../figures/SPA/SPA1_eff_vs_power.pdf')

### Run frequency scan

In [ ]:
# Define detunings
deltas = np.linspace(-10, 10, 101)*1e6 # in Hz

In [ ]:
# run simulations for different parameters
df_freq = pd.concat(Parallel(n_jobs = 9, verbose = 1)(delayed(parallel_func)(power=2e-4, detuning=delta, R0 = np.array([0,0,0.0]), N_steps=10000) for delta in deltas)).reset_index(drop=True)

In [ ]:
fig, ax = plt.subplots()
ax.plot(df_freq.detuning/1e6, df_freq.prob_fin, label = r'$|J=0, F = 0\rangle$')
ax.plot(df_freq.detuning/1e6, df_freq.prob_ini, label = r'$|J=1, m_J = 0, I_{tot} = 0\rangle$')
ax.set_xlabel(f"Microwave frequency - {muW_freq/1e6:.1f} / MHz")
ax.set_ylabel("Populations after SPA1")
ax.grid(True)
ax.legend(loc = 'lower left')
plt.savefig('../../../figures/SPA/SPA1_eff_vs_freq.pdf')

In [ ]:
df_freq['detuningMHz'] = df_freq.detuning/1e6
df_freq['powermW'] = df_freq.power*1e3

In [ ]:
# df_freq.to_csv(SAVE_DIR / Path('detuning_scan_with_bg_xypol_at_1e-4W_offset_E_field.csv'))